This notebook demonstrates the inference for text summarization using Falcon-7b-instruct model.

Getting inference mainly based on defining three steps: model, tokenizer, and pipeline.

In [ ]:
!pip install transformers
!pip install langchain

In [ ]:
import transformers
model = transformers.AutoModelForCausalLM.from_pretrained(
  'tiiuae/falcon-7b-instruct',
  trust_remote_code=True
)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [ ]:
from transformers import pipeline
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=100, do_sample=True, use_cache=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.1})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              Write a concise summary of the following text delimited by triple backquotes.
              ```{text}```
              SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')
import pandas as pd
test_df = pd.DataFrame(dataset['test'])

In [ ]:
test_df.head()
# test_df.drop(columns=['id'], inplace=True)


,article,highlights,id
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f


In [ ]:
test_df.drop(columns=['id'], inplace=True)

In [ ]:
test_df.head()

,article,highlights
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b..."
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...


In [ ]:
test_df['article'][0]

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony

In [ ]:
test_df['highlights'][0]

'Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .\nIsrael and the United States opposed the move, which could open the door to war crimes investigations against Israelis .'

In [ ]:
#generating the summary:

In [ ]:
# Create an empty column 'model_generated' in test_df to store the generated summaries
test_df['model_generated'] = ""

# Define a function to generate summaries and populate the 'model_generated' column
def generate_and_store_summary(row):
    article_text = row['article']
    summary = llm_chain.run(article_text)
    return summary

# Apply the generate_and_store_summary function to generate summaries for the first 25 articles
test_df.loc[:24, 'model_generated'] = test_df.loc[:24].apply(generate_and_store_summary, axis=1)

# Display the updated DataFrame with generated summaries for the first 25 articles
print(test_df[['article', 'model_generated']].head(25))


                                              article  \
0   (CNN)The Palestinian Authority officially beca...   
1   (CNN)Never mind cats having nine lives. A stra...   
2   (CNN)If you've been following the news lately,...   
3   (CNN)Five Americans who were monitored for thr...   
4   (CNN)A Duke student has admitted to hanging a ...   
5   (CNN)He's a blue chip college basketball recru...   
6   (CNN)Governments around the world are using th...   
7   (CNN)Andrew Getty, one of the heirs to billion...   
8   (CNN)Filipinos are being warned to be on guard...   
9   (CNN)For the first time in eight years, a TV l...   
10  London (CNN)A 19-year-old man was charged Wedn...   
11  (CNN)Paul Walker is hardly the first actor to ...   
12  (CNN)Seventy years ago, Anne Frank died of typ...   
13  (CNN)A year ago Bloomberg published a story wi...   
14  (CNN)If you're famous and performing the Ameri...   
15  (CNN)As goes Walmart, so goes the nation? Ever...   
16  (CNN)On May 28, 2014, some 

In [ ]:
test_df.head()

,article,highlights,model_generated
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,The Palestinian Authority becomes the 123rd m...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...","In Washington state, a dog named Theia was hi..."
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,Mohammad Javad Zarif is the Iranian Foreign M...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,CNN reported that five Americans who were exp...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,A Duke student admits to hanging a noose made...


Randomly visualization of generated text

In [ ]:
test_df['model_generated'][11]

' Paul Walker, an actor best known for his role in the "Fast and Furious" film franchise, has died in a tragic car wreck five years ago. Producers decided not to kill off his character in the upcoming blockbuster "Furious 7" and included Walker in the film despite his passing in 2013 with special effects and screen time for his surviving family members. Many fans are paying tribute to the actor through Facebook and other social media platforms, while the film premieres Wednesday.'

In [ ]:
test_df['model_generated'][24]

' Iran has been set to lift the ban on women at sports events but it is still unclear which sports they will be allowed to attend. A plan to allow "women and families" to enter sports stadiums will come into Effect in the next year. The ban was imposed in 1979 after Iran\'s Islamic Revolution and has been in place ever since. Iranian authorities deem mixed crowds un-Islamic. During a tournament in Australia this year, swooping coverage by Iranian fans caused the topic'

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

# Initialize the ROUGE evaluator
rouge = Rouge()

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate ROUGE scores for the selected samples
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge-1': {'r': 0.30021270994254523, 'p': 0.19448054685398838, 'f': 0.2269505079096782}, 'rouge-2': {'r': 0.0786281892161517, 'p': 0.04417277881644397, 'f': 0.0538095717629678}, 'rouge-l': {'r': 0.26353353456903017, 'p': 0.16879280331870816, 'f': 0.19795143143607846}}


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BLEU score for the selected samples
bleu_score = corpus_bleu(reference_summaries, generated_summaries)
print("BLEU Score for 25 Summaries:", bleu_score)


BLEU Score for 15 Summaries: 9.4726138403298e-232


In [ ]:
!pip install bert_score

In [ ]:
from bert_score import score
# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BERT Score
P, R, F1 = score(generated_summaries, reference_summaries, lang="en", verbose=True)

# Print BERT Score
print("BERT Precision:", P.mean().item())
print("BERT Recall:", R.mean().item())
print("BERT F1 Score:", F1.mean().item())


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.62 seconds, 15.48 sentences/sec
BERT Precision: 0.8189418911933899
BERT Recall: 0.8607068061828613
BERT F1 Score: 0.8385737538337708
